In [13]:
import os
from tqdm.auto import tqdm
from transformers import AutoTokenizer



In [ ]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [14]:
base_path = '/projets/melodi/gsantoss/complex-llm/generated-edoal'

In [28]:

result = {}

for p, d, fs in os.walk(base_path):
    for f in fs:
        spt = f.split('#')
        config = spt[4].split('.')[0]
        if config not in result:
            result[config] = {}

        if tuple(spt[1:3]) not in result[config]:
            result[config][tuple(spt[1:3])] = []

        result[config][tuple(spt[1:3])].append(os.path.join(p, f))

In [46]:
# result.pop('nq-s1-ns2')

errors = {}

for k, v in result.items():
    for kk, vv in v.items():
        for p in vv:

            with open(p, 'r') as f:
                txt = f.read()

                if '</rdf:RDF>' not in txt:
                    if k not in errors:
                        errors[k] = {}
                    if kk not in errors[k]:
                        errors[k][kk] = set()
                    errors[k][kk].add(p)


In [81]:
def is_valid_edoal(txt):
    return txt.endswith('</rdf:RDF>')


def can_repair(txt):
    return txt.rfind('<map>') > 0



for k, v in result.items():
    for kk, vv in v.items():

        edoals = []

        for p in vv:
            with open(p, 'r') as f:
                txt = f.read()

            if p in errors.get(k, {}).get(kk, set()):

                if not is_valid_edoal(txt):

                    if can_repair(txt):
                        last_map_index = txt.rfind('<map>')
                        repaired = txt[:last_map_index] + '\n\t</Alignment>\n</rdf:RDF>'
                        edoals.append(repaired)

                    else:
                        continue
            else:
                edoals.append(txt)
            
            
            
            
            
        if len(edoals) > 1:
            
            final_edoal = ''

        
            first = edoals[0]
            
            final_edoal += first[:first.find('<map>')]
            
            for e in edoals[1:]:
                final_edoal += e[e.find('<map>'):e.rfind('</map>')] + '\n\t</map>'
                
            final_edoal += '\n\t</Alignment>\n</rdf:RDF>'
            
        elif len(edoals) == 1:
            final_edoal = edoals[0]
            
        else:
            print(k, kk, 'No edoal files')
            
        
        out_path = f'/projets/melodi/gsantoss/complex-llm/final-edoal/{k}/'
        
        os.makedirs(out_path, exist_ok=True)
        with open(f'{out_path}{kk[0]}-{kk[1]}.edoal', 'w') as f:
            f.write(final_edoal)
            


nq-ns1-ns2 ('iasted', 'confOf') No edoal files
nq-ns1-ns2 ('cmt', 'sigkdd') No edoal files
nq-ns1-ns2 ('cmt', 'edas') No edoal files
nq-ns1-ns2 ('iasted', 'cmt') No edoal files
nq-ns1-ns2 ('confOf', 'cmt') No edoal files
nq-ns1-ns2 ('edas', 'iasted') No edoal files
nq-ns1-ns2 ('cmt', 'ekaw') No edoal files
nq-ns1-ns2 ('cmt', 'iasted') No edoal files
nq-ns1-ns2 ('confOf', 'ekaw') No edoal files
nq-ns1-ns2 ('iasted', 'conference') No edoal files
nq-ns1-ns2 ('iasted', 'edas') No edoal files
nq-ns1-ns2 ('conference', 'cmt') No edoal files
nq-ns1-ns2 ('conference', 'edas') No edoal files
nq-ns1-ns2 ('edas', 'sigkdd') No edoal files
nq-ns1-ns2 ('ekaw', 'iasted') No edoal files
nq-ns1-ns2 ('cmt', 'confOf') No edoal files
nq-ns1-ns2 ('sigkdd', 'cmt') No edoal files
nq-ns1-ns2 ('sigkdd', 'ekaw') No edoal files
nq-ns1-ns2 ('sigkdd', 'conference') No edoal files
nq-ns1-ns2 ('confOf', 'sigkdd') No edoal files
nq-ns1-ns2 ('confOf', 'iasted') No edoal files
nq-ns1-ns2 ('ekaw', 'confOf') No edoal fil